# Load the Documents

In [1]:
import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare import data_load

doc_loader = data_load.DocDataLoader()
config = Configuration()

folder = "test_training_program"
db = doc_loader.load_docs(data_folder=folder,size=800, overlap=40)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


From (8, 4) to Counter({1: 4, 2: 2, 4: 1, 3: 1})


# Choose model

### Gemini

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

INDEX = "gemini-major"
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=config.load_gemini_token())

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### VN SBert

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "vnsbert-major"

MODEL = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=conf.load_hg_token(0),
    model_name=MODEL)

### LaBSE

In [3]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
INDEX = "labse-uni"

MODEL = "sentence-transformers/LaBSE"

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=config.load_hg_token(0),
    model_name=MODEL)

# Push vector to  ES

In [4]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore

es = ElasticsearchStore.from_documents(db, embedding=embeddings,
                          es_connection=config.load_elasticsearch_connection(),
                          index_name=INDEX,
                          distance_strategy="EUCLIDEAN_DISTANCE")


# Push text data to ES

In [2]:
INDEX = "text-split-training_program"

k1 = 1.2
b = 0.75

# Define the index settings and mappings
settings = {
    "analysis": {"analyzer": {"default": {"type": "standard"}}},
    "similarity": {
        "custom_bm25": {
            "type": "BM25",
            "k1": k1,
            "b": b,
        }
    },
}
mappings = {
    "properties": {
        "content": {
            "type": "text",
            "similarity": "custom_bm25",  # Use the custom BM25 similarity
        }
    }
}

# Create the index with the specified settings and mappings
es_connect = config.load_elasticsearch_connection()
es_connect.indices.create(index=INDEX, mappings=mappings, settings=settings)
len(db)

15

In [3]:
from langchain.retrievers import ElasticSearchBM25Retriever
from src.custom.es_bm25_retriever import MyElasticSearchBM25Retriever
retriever = MyElasticSearchBM25Retriever(client=es_connect, index_name=INDEX)

In [4]:
a = retriever.add_documents(documents=db)

## Load

In [20]:
retriever.k = 2
a = retriever.get_relevant_documents("Hóa học")
a

[Document(page_content='Chương trình giáo dục đại học tiêu chuẩn có 40 chương trình:\n01. Bảo hộ lao động         21. Kỹ thuật cơ điện tử\n02. Công nghệ kỹ thuật môi trường (chuyên ngành Cấp thoát nước - Môi trường nước)         22. Quan hệ lao động\n03. Công nghệ sinh học         23. Quản lý thể dục thể thao – CN Kinh doanh thể thao và tổ chức sự kiện\n04. Công tác xã hội         24. Quản trị kinh doanh - CN Quản trị nguồn nhân lực\n05. Dược học         25. Marketing\n06. Kế toán         26. Quản trị kinh doanh - CN Quản trị Nhà hàng-Khách sạn\n07. Khoa học máy tính         27. Quy hoạch vùng và đô thị\n08. Khoa học môi trường         28. Tài chính - Ngân hàng\n09. Kiến trúc         29. Quản lý thể dục thể thao – chuyên ngành Golf\n10. Kinh doanh quốc tế         30. Thiết kế nội thất\n11. Kỹ thuật xây dựng         31. Thiết kế thời trang\n12. Kỹ thuật hóa học         32. Thiết kế đồ họa\n13. Kỹ thuật phần mềm         33. Thống kê\n14. Kỹ thuật xây dựng công trình giao thông         34

In [5]:
retriever.k = 1
a = retriever.get_relevant_documents("em muốn biết ngành kỹ thuật phần mềm")
a


[Document(page_content='Sinh viên còn được chú trọng tới việc nâng cao các kỹ năng nghề nghiệp cần thiết như: rèn luyện các phương pháp vẽ minh họa, vẽ kỹ thuật trong thiết kế thời trang, hướng dẫn các kỹ thuật cắt may từ đơn giản đến phức tạp. Số tiết học thực hành được thiết kế để cân bằng với số môn học lý thuyết và mang tính ứng dụng cao. Thông qua các đồ án chuyên ngành đa dạng và phong phú, sinh viên được học từ lý thuyết cơ bản đến khi có thể tự chuyển đổi ý tưởng thành sản phẩm hoàn thiện để trình diễn trước công chúng. Nhà trường còn tăng cường đầu tư phòng thực hành với không gian sáng tạo và các trang thiết bị chuyên dụng liên tục cập nhật, giúp sinh viên tiếp cận nhanh chóng với công nghệ và các xu hướng mới của ngành thời trang.\n\nĐiểm nổi bật của chương trình học là được xây dựng theo hướng kết nối với doanh nghiệp. Nhà trường thường xuyên kết hợp với các công ty trong và ngoài nước thuộc lĩnh vực thời trang để tổ chức các buổi tham quan, kiến tập, các cuộc thi thiết kế 